In [1]:
import matplotlib
matplotlib.use('Agg')
#%matplotlib notebook
%matplotlib inline
import sys
sys.settrace
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pylab import *
import struct
import array
import os
import glob
import h5py
from scipy.interpolate import griddata
from scipy import integrate

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
#plt.rcParams.update({
#    "text.usetex": True,
#    "font.family": "serif",
#    "font.serif": ["Palatino"],
#})

plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
plt.rcParams['font.size'] = '20'

from importlib import reload

/var/folders/2n/ftypqy6s65s2j7krblnn49cw0000gq/T/ipykernel_45521/772757402.py:30: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']


In [2]:
import athena_read

In [3]:
#This is the integral of blackbody from 0 to nu_t (h\nu/kT)
def FitBlackBody(nu_t):
  integral = 0.0;
  nu_2 = nu_t * nu_t;
  nu_3 = nu_t * nu_2;
  nu_7 = nu_2**2.0 * nu_3
  if nu_t < 1.8:
    integral = 0.051329911273422 * nu_3 -0.019248716727533 * nu_t * nu_3 + 0.002566495563671 * nu_2 * nu_3
    -3.055351861513195*1.e-5*nu_7;
  elif nu_t < 18.6:
    exp_nu = exp(-nu_t);
    integral = -0.156915538762850 * exp_nu * (nu_3 + 2.69 * nu_2 + 6.714 * nu_t) + 1.000009331428801*(1- exp_nu);
  else:
    integral = 1.0 - 192.1*exp(-0.9014*nu_t)

  return integral;
#This is the blackbody spectrum for each frequency nu_t (h\nu/kT)
def BlackBody(nu, tr):
  nu_t=nu/tr
  exp_1=exp(nu_t)-1
  if nu_t < 1.e-5:
    exp_1=tr**3.0*nu_t**2.0
  else:
    exp_1=nu**3.0/exp_1

  spec=(15/np.pi**4.0)*exp_1

  return spec;

In [65]:
def PlotProfile(datax, datay, xmin, xmax, ymin, ymax,  ylabel, label1, filename, xlabel='$r/r_g$', logscale=0, 
                xlogscale=0,datay1_2=None, datay1_3=None, datax2=None, datay2=None, datay2_2=None, datay2_3=None, 
                datax3=None, datay3=None, datay3_2=None, datay3_3=None, datax4=None, datay4=None, 
                datax5=None, datay5=None, label2='', label3='', label4='', label5='',title=None,leg_loc=None):
    plots, axes = plt.subplots(figsize=(9,9),dpi=300)
    plt.xlabel(xlabel, size = 30)
    plt.ylabel(ylabel, size = 30)
    plt.subplots_adjust(left=0.16,right=0.88,top=0.95,bottom=0.1)
    plt.ylim([ymin,ymax])
    plt.xlim([xmin,xmax])
    if logscale > 0:
      axes.set_yscale('log')
    if xlogscale > 0:
      axes.set_xscale('log')
    if title is not None:
      plt.title(title,size=20)

    plt.plot(datax,datay,color='black',marker=' ',fillstyle='none',markersize=8,label=label1,linewidth=2.0,alpha=1.0)
    if datay1_2 is not None:
      plt.scatter(datax,datay1_2,color='red',marker='o',s=40)
    if datay1_3 is not None:
      plt.plot(datax,datay1_3,color='black',linestyle='dashed',linewidth=4.0)
    if datay2 is not None:
      plt.plot(datax2,datay2,color='red',label=label2,linewidth=2.0,alpha=1.0)
    if datay2_2 is not None:
      plt.plot(datax2,datay2_2,color='black',linestyle='dashed',linewidth=2.0,alpha=1.0)
    if datay2_3 is not None:
      plt.plot(datax2,datay2_3,color='red',linestyle='dashed',linewidth=4.0)
    if datay3 is not None:
      plt.scatter(datax3,datay3,edgecolor='black',marker='s',s=50,facecolors='none')
    if datay3_2 is not None:
      plt.plot(datax3,datay3_2,color='black',linestyle='dashed',linewidth=2.0,alpha=1.0)
    if datay3_3 is not None:
      plt.plot(datax3,datay3_3,color='green',linestyle='dashed',linewidth=4.0)
    if datay4 is not None:
      plt.scatter(datax4,datay4,edgecolor='red',marker='s',s=50,facecolors='none')
    if datay5 is not None:
      plt.plot(datax5,datay5,color=tableau20[0],label=label5,linewidth=2.0) 
    if leg_loc is not None:
      plt.legend(loc="best",bbox_to_anchor=leg_loc,frameon=False)
    axes.set_aspect('auto')
#    axes.yaxis.set_tick_params(labelsize=25)
#    axes.xaxis.set_tick_params(labelsize=25)
    plt.savefig(filename)
    plt.close(plots)


In [15]:
files=sorted(glob.glob('Data/thermal.out4.*athdf'))
num_file=len(files)

In [16]:
#for filename in files:
ang_file = open('Data/Rad_angles.txt', 'r')
Lines = ang_file.readlines()
angle_line = [ line for line in Lines if  "tau_scheme" in line]
angle_line=angle_line[0]
crat_line = [ line for line in Lines if  "Crat" in line]
crat_line=crat_line[0]
Crat_split=crat_line.split(" ")
Crat=float(Crat_split[size(Crat_split)-2])
angle_index=Lines.index(angle_line)+1
location=Lines.index('fre   spec\n')
tot_line=size(Lines)
nfreq=tot_line-location-1
nu_grid=np.zeros(nfreq)
histories=np.zeros((num_file,52))
for i in range(nfreq):
    line=Lines[i+location+1].split(' ')[0]
    nu_grid[i]=float(line)

nu_center=np.zeros(nfreq-1)
nu_center=(nu_grid[:-1]+nu_grid[1:])/2
#nu_center=np.append(nu_center,nu_grid[nfreq-1])
nang=location-angle_index
mu_x=np.zeros(nang)
mu_y=np.zeros(nang)
mu_z=np.zeros(nang)
weight=np.zeros(nang)
for n in range(nang):
    line=Lines[n+angle_index].split('   ')
    mu_x[n]=float(line[1])
    mu_y[n]=float(line[2])
    mu_z[n]=float(line[3])
    weight[n]=float(line[4].rstrip())

In [19]:
Prat=0
#Crat=8.0534e4
count=0
filename=files[num_file-1]
with h5py.File(filename, 'r') as f:
  attributes = f.attrs.items()
  attrs = dict(attributes)
  level = f.attrs['MaxLevel']
  time = f.attrs['Time']
  subsample = False

data = athena_read.athdf(filename, level=level, subsample=subsample)
vx=np.average(data['vel1'])
lorz=1.0/(1-vx**2.0/Crat**2.0)**0.5

Er_spec=np.zeros(nfreq-1)
Er0_spec=np.zeros(nfreq-1)
for i in range(nfreq-1):
    varname='Er_'+str(i)
    Er_spec[i]=np.average(data[varname])/(nu_grid[i+1]-nu_grid[i])
    varname0='Er0_'+str(i)
    Er0_spec[i]=np.average(data[varname0])/(nu_grid[i+1]-nu_grid[i])    
#tgas=np.average(data['press']/data['rho'])
tr=1.0
#blackbody=np.zeros(nfreq)
#for i in range(nfreq-1):
#    kt_right=nu_grid[i+1]/tgas
#    kt_left=nu_grid[i]/tgas
#    blackbody[i]=tgas**4.0*(FitBlackBody(kt_right)-FitBlackBody(kt_left))
#blackbody[nfreq-1]=tgas**4.0*(1-FitBlackBody(nu_grid[nfreq-1]/tgas))
bd_bins=100
blackbody=np.zeros(bd_bins)
blackbody_cm=np.zeros(bd_bins)
nu_bd=np.linspace(1.e-5,20,bd_bins)
for i in range(bd_bins):
    blackbody[i]=BlackBody(nu_bd[i],tr)
weight_cm=np.zeros(nang)
for n in range(nang):
    vdotn=vx*mu_x[n]
    vnc=1-vdotn/Crat
    tran_coef=lorz * vnc
    weight_cm[n]=weight[n]/tran_coef**2.0

sum_weight=np.sum(weight_cm)
weight_cm[:]=weight_cm[:]/sum_weight
bd_intensity_cm=np.zeros((nang,bd_bins))
for i in range(bd_bins):
    blackbody_cm[i]=0.0
    for n in range(nang):
        vdotn=vx*mu_x[n]
        vnc=1-vdotn/Crat
        tran_coef=lorz * vnc
        nu_lab=nu_bd[i]/tran_coef
        blackbody_cm[i]=blackbody_cm[i]+tran_coef**3.0*BlackBody(nu_lab,tr)*weight_cm[n]
        bd_intensity_cm[n,i]=tran_coef**3.0*BlackBody(nu_lab,tr)

In [20]:
print(np.shape(bd_intensity_cm))

(4, 100)


In [10]:
###Now load data for specific intensity
files=sorted(glob.glob('Data/thermal.out5.*athdf'))
num_file=len(files)
filename=files[num_file-1]
with h5py.File(filename, 'r') as f:
  attributes = f.attrs.items()
  attrs = dict(attributes)
  level = f.attrs['MaxLevel']
  time = f.attrs['Time']
  subsample = False

data2 = athena_read.athdf(filename, level=level, subsample=subsample)

In [11]:
intensity=np.zeros((nfreq-1,nang))
count=0
for ifr in range(nfreq-1):
    for n in range(nang):
        var_name='user_out_var'+str(count)
        intensity[ifr,n]=np.average(data2[var_name])
        count=count+1

In [12]:
ir_unshift=np.zeros((nfreq-1,nang))
nu_shift=np.zeros((nfreq-1,nang))
for ifr in range(nfreq-1):
    for n in range(nang):
        tran_coef=lorz*(1-vx*mu_x[n]/Crat)
        ir_unshift[ifr,n]=Er_spec[ifr]*(nu_grid[ifr+1]-nu_grid[ifr])*tran_coef**4.0
        nu_shift[ifr,n]=nu_center[ifr]*tran_coef
#get monochromatic intensity
for ifr in range(nfreq-1):
    for n in range(nang):
        tran_coef=lorz*(1-vx*mu_x[n]/Crat)
        delta_nu=nu_grid[ifr+1]-nu_grid[ifr]
        ir_unshift[ifr,n]=ir_unshift[ifr,n]/(tran_coef*delta_nu)
        intensity[ifr,n]=intensity[ifr,n]/delta_nu

In [13]:
ylabel='$\\rm Blackbody Spectrum E_r(\\nu)$'
filename='blackbody_spectrum.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_bd,blackbody,2.e-3, 50, 1.e-8, 1, ylabel, None, filename, xlabel, logscale=1,
            datax2=nu_center,datay2=Er_spec,xlogscale=1)
ylabel='$\\rm Blackbody Spectrum E_{r,0}(\\nu)$'
filename='blackbody_spectrum_cm.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_bd,blackbody_cm,2.e-3, 50, 1.e-8, 1, ylabel, None, filename, xlabel, logscale=1,
            datax2=nu_center,datay2=Er0_spec,xlogscale=1)

In [13]:
ylabel='$\\rm E_r(\\nu) - E_{r,0}(\\nu)$'
filename='diff_spectrum.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_bd,blackbody-blackbody_cm,2.e-3, 50, -5.e-3,1.e-3, ylabel, None, filename, xlabel, logscale=0,
            datax2=nu_center,datay2=Er_spec-Er0_spec,xlogscale=1)

In [14]:
##intensity for different angles
ylabel='$\\rm I_{\\nu,n=0}$'
filename='intensity_spectrum.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_center,intensity[:,0],2.e-3, 50, 1.e-5, 1, ylabel, None, filename, xlabel, logscale=1,
            datax2=nu_shift[:,0],datay2=ir_unshift[:,0],xlogscale=1)


In [15]:
##intensity for different angles
ylabel='$\\rm I_{\\nu,n=1}$'
filename='intensity_spectrum1.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_center,intensity[:,1],2.e-3, 50, 1.e-5, 1, ylabel, None, filename, xlabel, logscale=1,
            datax2=nu_shift[:,1],datay2=ir_unshift[:,1],xlogscale=1)

In [16]:
##intensity for different angles
ylabel='$\\rm I_{\\nu,n=2}$'
filename='intensity_spectrum2.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_center,intensity[:,2],2.e-3, 50, 1.e-5, 1, ylabel, None, filename, xlabel, logscale=1,
            datax2=nu_shift[:,2],datay2=ir_unshift[:,2],xlogscale=1)

In [66]:
ylabel='$I_{0,\\nu}$'
filename='check_intensity.pdf'
xlabel='$\\tilde{\\nu}$'

#print(histories[:,0])
PlotProfile(nu_bd,bd_intensity_cm[0,:],3.e-2, 30, 5.e-5, 1, ylabel, '$n\\cdot v>0$', filename, xlabel, logscale=1,
            datax2=nu_bd,datay2=bd_intensity_cm[1,:],label2='$n\\cdot v<0$',
            datax3=nu_center,datay3=intensity[:,0],
            datax4=nu_center,datay4=intensity[:,1],xlogscale=1,leg_loc=(0.35, 0.3))

In [18]:
np.sum(intensity[0,:]*weight_cm)

0.0005342949056536798

In [64]:
vx/Crat

0.134